# Importes

In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import slate3k as slate
import nltk
from lxml import etree
from urllib.request import urlopen

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/p1ntsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Funções e classes

In [4]:
class RequestBeautifulSoup:
    def __init__(self,url):
        self.url = url
        self.intervalos = None
        self.perguntas = None
        self.respostas = None
    
    # Request do soup da biblioteza BeautifulSoup
    def requestSoup(self):
        response = get(self.url)
        return BeautifulSoup(response.text, 'html.parser')
    
    # Intervalos de onde estão as respostas
    def getIntervalos(self,listaPrincipal):
        intervalos = []
        for i in range(len(self.perguntas[1])):
            if(i != len(self.perguntas[1])-1):
                intervalos.append((self.perguntas[1][i]+1,self.perguntas[1][i+1]))
            else:
                intervalos.append((self.perguntas[1][i]+1,len(listaPrincipal)))
        self.intervalos = intervalos

    # Perguntas e suas posições
    def getPerguntas(self,soup, chaves = None):
        perguntas = []
        posi = []
        for i in range(len(soup)):
            if("?" in soup[i].text):
                if(chaves is not None):
                    texto = soup[i].text
                    for j in chaves:
                        texto = texto.replace(j,"")
                    perguntas.append(texto)
                    posi.append(i)
                else:
                    perguntas.append(soup[i].text)
                    posi.append(i)

        self.perguntas = [perguntas,posi]
    
    # Respostas
    def getRespostas(self,soup,chaves = None,subs = ""):
        respostas = []
        for i in self.intervalos:
            text = ""
            for j in range(i[0],i[1]):
                text += soup[j].text
            if(chaves is not None):
                for j in chaves:
                    text = text.replace(j,subs)
                respostas.append(text)
            else:
                respostas.append(text)
        self.respostas = respostas
        
    def getComparacao(self,first,second):
        if(len(first) == len(second)):
            return True
        else:
            return False
        
    def criarDataframePDF(self,perguntas,respostas,salve = None):
        if(salve is not None):    
            pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./' + salve + '.csv', index= False)
        else:
            return pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas})
    

In [78]:


def getXpathWay(url):
    response = urlopen(url)
    htmlparser = etree.HTMLParser()
    return etree.parse(response, htmlparser)
        

def getPerguntasXpath(soup, chaves = None):
    perguntas = []
    posi = []
    for i in range(len(soup)):
        if("?" in soup[i]):
            if(chaves is not None):
                texto = soup[i]
                for j in chaves:
                    texto = texto.replace(j,"")
                perguntas.append(texto)
                posi.append(i)
            else:
                perguntas.append(soup[i])
                posi.append(i)
                
    return [perguntas,posi]
                
def getPerguntasPDF(soup, chaves = None):
    perguntas = []
    posi = []
    for i in range(len(soup)):
        if("?" in soup[i]):
            if(chaves is not None):
                texto = soup[i]
                for j in chaves:
                    texto = texto.replace(j,"")
                perguntas.append(texto)
                posi.append(i)
            else:
                perguntas.append(soup[i])
                posi.append(i)
    return [perguntas,posi]

def getRespostasPDF(soup,intervalos,chaves = None):
    respostas = []
    for i in intervalos:
        text = ""
        for j in range(i[0],i[1]):
            text += soup[j]
        for k in chaves:
            text = text.replace(k,"")
        respostas.append(text)
    return respostas

# Intervalos de onde estão as respostas
def getIntervalos(posicoes,listaPrincipal):
    intervalos = []
    for i in range(len(posicoes)):
        if(i != len(posicoes)-1):
            intervalos.append((posicoes[i]+1,posicoes[i+1]))
        else:
            intervalos.append((posicoes[i]+1,len(listaPrincipal)))
    return intervalos

def criarDataframePDF(perguntas,respostas,salve = None):
    if(salve is not None):    
        pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./' + salve + '.csv', index= False)
    else:
        return pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas})

# Código

In [38]:
objeto_soup = RequestBeautifulSoup('https://www.anchieta.es.gov.br/pagina/ler/2063/o-que-e-a-covid-19-duvidas-e-orientacoes')
soup = objeto_soup.requestSoup()

In [39]:
div_soup = soup.find('div', class_ = 'panel-body auto-caption')
list_soup = div_soup.find_all(['p','ul','il','a','h2','h3'])

In [40]:
list_soup

[<p><h2><strong><span style="color:#27ae60">O que é COVID-19</span></strong></h2>
 <p>A <strong>COVID-19</strong> é uma doença causada pelo coronavírus <strong>SARS-CoV-2</strong>, que apresenta um quadro clínico que varia de infecções assintomáticas a quadros respiratórios graves. De acordo com a Organização Mundial de Saúde (OMS), a maioria dos pacientes com COVID-19 (cerca de 80%) podem ser assintomáticos e cerca de 20% dos casos podem requerer atendimento hospitalar por apresentarem dificuldade respiratória e desses casos aproximadamente 5% podem necessitar de suporte para o tratamento de insuficiência respiratória (suporte ventilatório).</p>
 <h3><strong><span style="color:#27ae60">O que é o coronavírus?</span></strong></h3>
 <h3>Coronavírus é uma família de vírus que causam infecções respiratórias. <strong>O novo agente do coronavírus foi descoberto em 31/12/19</strong> após casos registrados na China. Provoca a doença chamada de coronavírus (COVID-19).</h3>
 <p>Os primeiros coro

In [41]:
objeto_soup.getPerguntas(list_soup)
objeto_soup.getIntervalos(list_soup)
objeto_soup.getRespostas(list_soup,['\xa0','\n'])

In [42]:
print(objeto_soup.perguntas[0])
print(objeto_soup.respostas)
print(objeto_soup.getComparacao(objeto_soup.perguntas[0],objeto_soup.respostas))

['O que é COVID-19\nA\xa0COVID-19\xa0é uma doença causada pelo coronavírus\xa0SARS-CoV-2, que apresenta um quadro clínico que varia de infecções assintomáticas a quadros respiratórios graves. De acordo com a Organização Mundial de Saúde (OMS), a maioria dos pacientes com COVID-19 (cerca de 80%) podem ser assintomáticos e cerca de 20% dos casos podem requerer atendimento hospitalar por apresentarem dificuldade respiratória e desses casos aproximadamente 5% podem necessitar de suporte para o tratamento de insuficiência respiratória (suporte ventilatório).\nO que\xa0é o coronavírus?\nCoronavírus é uma família de vírus que causam infecções respiratórias.\xa0O novo agente do coronavírus foi descoberto em 31/12/19\xa0após casos registrados na China. Provoca a doença chamada de coronavírus (COVID-19).\nOs primeiros coronavírus humanos foram isolados pela primeira vez em 1937. No entanto, foi em 1965 que o vírus foi descrito como coronavírus, em decorrência do perfil na microscopia, parecendo 

In [43]:
objeto_soup.criarDataframePDF(objeto_soup.perguntas[0],objeto_soup.respostas)

,Perguntas,Respostas
0,O que é COVID-19\nA COVID-19 é uma doença caus...,O que é COVID-19ACOVID-19é uma doença causada ...
1,O que é o coronavírus?,Coronavírus é uma família de vírus que causam ...


In [87]:
with open('../img/1.pdf','rb') as f:
    extract = slate.PDF(f)[0]

In [88]:
extract.split('\n\n')

['Algumas pessoas são infectadas mas não desenvolvem sintoma algum e não se sentem mal. A ',
 'maioria das pessoas (cerca de 80%) se recupera da doença sem a necessidade de nenhum ',
 'tratamento especial. Os casos mais graves desenvolvem dificuldade de respirar.  ',
 'Pessoas idosas, aquelas com o sistema imunológico enfraquecido ou problemas de saúde ',
 'preexistentes, como hipertensão, problemas cardíacos ou diabetes, estão mais suscetíveis a ',
 'adoecer gravemente. Quem \natendimento médico. ',
 'tiver ',
 'febre, ',
 'tosse e dificuldade para ',
 'respirar deve procurar ',
 'O COVID-19 é um vírus novo e não se sabe ao certo como interage com Chagas, pois não temos ',
 'evidências científicas suficientes. É certo que aqueles que já têm problemas cardíacos graves ',
 'causados pela doença de Chagas ou um sistema imunológico enfraquecido correm mais risco ',
 'de desenvolver sintomas mais graves de COVID-19. ',
 'Se sou paciente crônico de doença de Chagas sem complicações graves, 

In [84]:
perguntas = ['Quais são os sintomas de COVID-19? São diferentes para um paciente de doença de Chagas?','Se sou paciente crônico de doença de Chagas sem complicações graves, sou mais vulnerável ao COVID-19?','Se estou na fase aguda da doença de Chagas e tenho sintomas de COVID- 19, o que devo fazer?','Se eu tiver sintomas ou o resultado do teste de COVID-19 for positivo, preciso interromper o tratamento de doença de Chagas?','O que devo fazer se eu tiver doença de Chagas sem tratamento específico e tiver sintomas de COVID-19?','O que devo fazer para proteger minha família e a mim do COVID-19 se tenho doença de Chagas?','Se eu tiver complicações cardiológicas ou digestivas causadas pela doença de Chagas, o que preciso fazer para me proteger do COVID-19?','Devo continuar frequentando o centro de saúde para dar continuidade ao tratamento de Chagas durante a epidemia de COVID-19?','Se eu receber um resultado positivo para COVID-19 e tiver doença de Chagas, quais são os tratamentos que não devo tomar?','Se estou grávida e suspeito que tenha contraído doença de Chagas, sou mais vulnerável ao COVID-19?','Se eu estiver grávida, tiver doença de Chagas e receber um resultado positivo para COVID-19, o que devo fazer?']

In [85]:
perguntas

['Quais são os sintomas de COVID-19? São diferentes para um paciente de doença de Chagas?',
 'Se sou paciente crônico de doença de Chagas sem complicações graves, sou mais vulnerável ao COVID-19?',
 'Se estou na fase aguda da doença de Chagas e tenho sintomas de COVID- 19, o que devo fazer?',
 'Se eu tiver sintomas ou o resultado do teste de COVID-19 for positivo, preciso interromper o tratamento de doença de Chagas?',
 'O que devo fazer se eu tiver doença de Chagas sem tratamento específico e tiver sintomas de COVID-19?',
 'O que devo fazer para proteger minha família e a mim do COVID-19 se tenho doença de Chagas?',
 'Se eu tiver complicações cardiológicas ou digestivas causadas pela doença de Chagas, o que preciso fazer para me proteger do COVID-19?',
 'Devo continuar frequentando o centro de saúde para dar continuidade ao tratamento de Chagas durante a epidemia de COVID-19?',
 'Se eu receber um resultado positivo para COVID-19 e tiver doença de Chagas, quais são os tratamentos que n